# New section

In [ ]:

!pip install ultralytics
!pip install deep-sort-realtime opencv-python

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab.patches import cv2_imshow  # <-- Needed for Colab

# ============================
# LOAD YOUR YOLO PERSON MODEL
# ============================

PERSON_MODEL_PATH = "/content/drive/MyDrive/person detection/yolov8n.pt"
person_model = YOLO(PERSON_MODEL_PATH)

# ============================
# INITIALIZE DeepSORT
# ============================

tracker = DeepSort(
    max_age=25,
    n_init=2,
    max_iou_distance=0.7,
    embedder="mobilenet",
    half=True,
)

# ============================
# PROCESS INPUT VIDEO
# ============================

input_video_path = "/content/drive/MyDrive/person detection/PETS09-S2L2-raw.mp4"
output_video_path = "/content/drive/MyDrive/person detection/2_output_tracked.mp4"

cap = cv2.VideoCapture(input_video_path)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

print("Processing...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # ============================
    # YOLO INFERENCE
    # ============================
    results = person_model(frame, conf=0.45)[0]

    detections = []

    for box in results.boxes:
        cls_id = int(box.cls[0].item())  # FIXED
        label = person_model.names[cls_id]

        if label != "person":
            continue

        x1, y1, x2, y2 = box.xyxy[0].tolist()  # FIXED
        conf = float(box.conf[0].item())      # FIXED

        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, label))

    # ============================
    # DeepSORT TRACKING
    # ============================
    tracks = tracker.update_tracks(detections, frame=frame)

    # ============================
    # DRAW RESULTS
    # ============================

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # ============================
    # SHOW REAL-TIME OUTPUT (Colab)
    # ============================
    cv2_imshow(frame)  # <-- This works in Colab

    out.write(frame)

cap.release()
out.release()

print("Processing complete! File saved at:", output_video_path)
